In [ ]:
import pandas as pd
import numpy as np
###list of data files
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time
import math
from itertools import groupby
from operator import itemgetter
import more_itertools as mit
import time
import datetime

In [ ]:
from dtw import *
### Dynamic Time Warping(DTW) method
def DTWDistance(s1, s2):
    alignment = dtw(s1,s2, keep_internals=True)
    d = alignment.normalizedDistance
    return d

In [ ]:
def transfer_to_list_of_pd(clusters_list):
    clusters_data_list = []
    for i in clusters_list:
        temp_data = pd.DataFrame(i, columns =['SecurityID', 'Sector', 'Industry','Sub_Industry','Seniority','Issuedby','Country','Rating','AmountIssued','Coupon','IssueDate','MaturityDate','ModifiedDuration'])
        clusters_data_list.append(temp_data)
    return clusters_data_list

In [ ]:
def assign_rating(x):
    x = float(x)
    if x < 0:
        return 'NR'
    elif x >0 and x<=0.5:
        return 'D'
    elif x > 0.5 and x<=1.5:
        return 'C'
    elif x > 1.5 and x<=2.5:
        return 'CC-'
    elif x > 2.5 and x<=3.5:
        return 'CC'
    elif x >3.5 and x<=4.5:
        return 'CC+'
    elif x >4.5 and x <=5.5:
        return 'CCC-'
    elif x >5.5 and x <=6.5:
        return 'CCC'
    elif x >6.5 and x<=7.5:
        return 'CCC+'
    elif x >7.5 and x <= 8.5:
        return 'B-'
    elif x>8.5 and x <=9.5:
        return 'B'
    elif x> 9.5 and x <= 10.5:
        return 'B+'
    elif x >10.5 and x<=11.5:
        return 'BB-'
    elif x > 11.5 and x<=12.5:
        return 'BB'
    elif x >12.5 and x<=13.5:
        return 'BB+'
    elif x>13.5 and x <=14.5:
        return 'BBB-'
    elif x>14.5 and x<=15.5:
        return 'BBB'
    elif x>15.5 and x<=16.5:
        return 'BBB+'
    elif x>16.5 and x<=17.5:
        return 'A-'
    elif x > 17.5 and x<=18.5:
        return 'A'
    elif x >18.5 and x<=19.5:
        return 'A+'
    elif x>19.5 and x<=20.5:
        return 'AA-'
    elif x >20.5 and x<=21.5:
        return 'AA'
    elif x>21.5 and x<=22.5:
        return 'AA+'
    else:
        return 'AAA'

In [ ]:
def process_rating_data(data):
    #data = data[data['KeyDate']<=end_date]
    rating = data.groupby('SecurityID',as_index=False).mean()
    rating['RatingSP'] = rating['RatingSP'].apply(lambda x:assign_rating(x))
    return rating

In [ ]:
def grab_clusters(clusters,bond_spread_list,security_data,rating_data):
    temp_list = []
    for cluster in clusters:
        temp_list2 = []
        for bond_index in cluster:
            bond_id = bond_spread_list[bond_index][1]
            bond_Sector = security_data[security_data['SecurityID']==bond_id].SecuritySector.iloc[0]
            bond_Industry = security_data[security_data['SecurityID'] == bond_id].SecurityIndustry.iloc[0]
            bond_sub_Industry = security_data[security_data['SecurityID'] == bond_id].SecuritySubIndustry.iloc[0]
            bond_Seniority = security_data[security_data['SecurityID'] == bond_id].Seniority.iloc[0]
            bond_Issuer =security_data[security_data['SecurityID'] == bond_id].Issuer.iloc[0]
            bond_countryrisk =security_data[security_data['SecurityID'] == bond_id].CountryRisk.iloc[0]
            bond_amountIssued =security_data[security_data['SecurityID'] == bond_id].AmountIssued.iloc[0]
            bond_coupon =security_data[security_data['SecurityID'] == bond_id].Coupon.iloc[0]
            bond_MaturityDate =security_data[security_data['SecurityID'] == bond_id].MaturityDate.iloc[0]
            bond_issuedate =security_data[security_data['SecurityID'] == bond_id].IssueDate.iloc[0]
            bond_duration =security_data[security_data['SecurityID'] == bond_id].ModifiedDuration.iloc[0]
            try:
                bond_rating=rating_data[rating_data['SecurityID'] == bond_id].RatingSP.iloc[0]
            except:
                bond_rating = 'Not Founded'
            temp_tuple = (bond_id,bond_Sector,bond_Industry,bond_sub_Industry,bond_Seniority,bond_Issuer,bond_countryrisk,bond_rating,bond_amountIssued,bond_coupon,bond_issuedate,bond_MaturityDate,bond_duration)
            temp_list2.append(temp_tuple)
        temp_list.append(temp_list2)
    return temp_list

In [ ]:
def merge_left_right(data1,data2):
    temp_data = data1.append(data2)
    return temp_data

def merg_sort(list_data):
    list_length = len(list_data)
    if list_length == 1:
        output = list_data[0][2]
        return output
    else:
        mid_point = math.floor(list_length/2)
        #print(len(list_data[:mid_point]))
        left = list_data[mid_point:]
        right = list_data[:mid_point]
        return merge_left_right(merg_sort(left), merg_sort(right))

In [ ]:
def sliding_windows(data,period_of_time,start_date):
    #transfer start_date to datetime()
    time_list = start_date.split('-')
    year = int(time_list[0])
    month = int(time_list[1])
    date = int(time_list[2])
    start_datetime = datetime.datetime(year, month, date)
    data['KeyDate'] = pd.to_datetime(data['KeyDate'])
    data['MaturityDate'] = pd.to_datetime(data['MaturityDate'])
    data['IssueDate'] = pd.to_datetime(data['IssueDate'])
    ###find the end date:
    end_datetime = start_datetime + datetime.timedelta(days=period_of_time)
    end_date = str(end_datetime.date())
    #####select data before the end_date
    data = data[data['KeyDate'] >= start_datetime]
    data = data[data['KeyDate'] <= end_date]
    data = data[data['IssueDate'] <= start_datetime]
    data = data[data['MaturityDate'] >= end_date]
    return data

In [ ]:
def apply_fill(data_list,index):
    for i in data_list:
        try:
            fill_the_GSpeard(i)
        except:
            index.append(i)
    return index,data_list

In [ ]:
def merge_data(lists,initial_data):
    data = initial_data
    for file in lists:
        path = 'C:\\Users\\y437l\\OneDrive\\MMAI\\Capstone\\Data\\{}'.format(file)
        temp_data = pd.read_csv(path)
        print(len(temp_data))
        data = data.append(temp_data)
        print(len(data))
    return data

In [ ]:
start_time = time.time()
security_data = pd.read_csv("C:/Users/y437l/OneDrive/MMAI/Capstone/Data/SecurityData - Copy-LAPTOP-OAEJRPE8.csv")
security_data =security_data[['SecurityID','Currency','IssueDate','MaturityDate']]
### merge spread data togther
sc = MinMaxScaler()
data = pd.read_csv("C:/Users/y437l/OneDrive/MMAI/Capstone/Data/1.csv")
file_lists = ['14426.csv','24001.csv','36128.csv','48087.csv','55086.csv']
final_data = merge_data(file_lists,data)
#final_data = data
###left join the currency data into the spread data
final_data = final_data.merge(security_data, on=['SecurityID'], how='left')
final_data.dropna(subset=["ZSpread"],inplace=True)
###select currency as USD
final_data_1 = final_data[final_data.Currency == 'USD']
final_data = final_data_1.groupby('SecurityID')
###create a list of bond with data
bonds_list = [final_data.get_group(x) for x in final_data.groups]

In [ ]:
def fill_the_GSpeard(data):
    ##calucate the spread between gspread and z spread if the gspread is not 0.
    data['Spread_G_Z'] = data.GSpread - data.ZSpread
    ###find the nan data's index in the data:
    index_with_indicator = data.GSpread.isna()
    index_with_indicator = index_with_indicator.tolist()
    index_list = []
    for indx in range(0,len(index_with_indicator)):
        if index_with_indicator[indx] == True:
            index_list.append(indx)
    #print(index_list)
    ##find concustive missing value:
    ##example:[(22, 39), 303, (326, 343), 607]
    star_end_dates = find_ranges(index_list)
    missing_date_dict={}
    for periods in star_end_dates:
        if type(periods) !=tuple:
            missing_date_dict[str(periods)] = spread_slope(data,periods)
            value = data.Spread_G_Z.iloc[periods-1]+missing_date_dict[str(periods)][3]
            print(value)
            data.GSpread.iloc[periods] = data.ZSpread.iloc[periods]+value
            print(data.GSpread.iloc[periods])
        else:
            missing_date_dict['{}-{}'.format(periods[0],periods[1])] = spread_slope(data, periods)
            star = missing_date_dict['{}-{}'.format(periods[0],periods[1])][0]
            end = missing_date_dict['{}-{}'.format(periods[0],periods[1])][1]
            slope = missing_date_dict['{}-{}'.format(periods[0],periods[1])][3]
            for days in range(star+1,end):
                value = data.Spread_G_Z.iloc[star] + (days-star)*slope
                data.GSpread.iloc[days] = data.ZSpread.iloc[days]+value
    print(missing_date_dict)
    #if missing_date_dict['{}-{}'.format(periods[0],periods[1])][3] >=1:
        #print(missing_date_dict)
        #print("Warning {}".format(data.SecurityID.iloc[0]))
    return data

In [ ]:
def find_ranges(iterable):
    """Yield range of consecutive numbers."""
    for group in mit.consecutive_groups(iterable):
        group = list(group)
        if len(group) == 1:
            yield group[0]
        else:
            yield group[0], group[-1]

In [ ]:
def spread_slope(data,input):
    if type(input) != tuple:
        star = input-1
        end = input+1
    else:
        star = input[0] - 1
        end = input[1] + 1
    if end == len(data):
        slope = data.Spread_G_Z.iloc[star]
        end = 'None'
        gap = 1
    else:
        print(input)
        star_point = data.Spread_G_Z.iloc[star]
        print(star_point)
        end_point = data.Spread_G_Z.iloc[end]
        print(end_point)
        gap = (end-star)
        print(gap)
        slope = (end_point - star_point) / gap
        print(slope)
    return (star, end, gap,slope)

In [ ]:
def sliding_windows(data,period_of_time,start_date):
    #transfer start_date to datetime()
    time_list = start_date.split('-')
    year = int(time_list[0])
    month = int(time_list[1])
    date = int(time_list[2])
    start_datetime = datetime.datetime(year, month, date)
    data['KeyDate'] = pd.to_datetime(data['KeyDate'])
    data['MaturityDate'] = pd.to_datetime(data['MaturityDate'])
    data['IssueDate'] = pd.to_datetime(data['IssueDate'])
    ###find the end date:
    end_datetime = start_datetime + datetime.timedelta(days=period_of_time)
    end_date = str(end_datetime.date())
    #####select data before the end_date
    data = data[data['KeyDate'] <= end_date]
    data = data[data['IssueDate'] <= start_datetime]
    data = data[data['MaturityDate'] >= end_date]
    return data


In [ ]:
def select_data(num,listoftuble):
    temp_list = []
    for i in listoftuble:
        if i[0] == num:
            temp_list.append(i)
    return temp_list

## Select the data with only spread percentage change.
def select_spread_data(listoftuble):
    temp_list = []
    for i in listoftuble:
        temp_list.append(i[2])
    return temp_list

In [ ]:
problemtic_data_index =[]
problem_data_index,bonds_list = apply_fill(bonds_list,problemtic_data_index)
print("--- %s seconds ---" % (time.time() - start_time),'Next Step:Percentage Changed')

In [ ]:
print(len(bonds_list))

In [ ]:
bond_spread_list = []
for bond in bonds_list:
    try:
        bond['avg_GSpread'] = bond['GSpread'].rolling(window=10).mean()
        bond.dropna(subset=['avg_GSpread'],inplace=True)
        bond = sliding_windows(bond, 90, '2019-01-15')
        if "-" not in bond.G_change.values:
            bond_spread_list.append((len(bond.GSpread.values),bond.SecurityID.iloc[0],bond))
        else:
            print(bond.SecurityID.iloc[0])
    except:
        pass

In [ ]:
bond_spread_list[1]

In [ ]:
sc = MinMaxScaler()

In [ ]:
new_data = merg_sort(bond_spread_list)
new_data = new_data[new_data["avg_GSpread"]!="-"]

In [ ]:
new_data

In [ ]:
new_data['G_scale'] = sc.fit_transform(new_data[['avg_GSpread']], y=None)

In [ ]:
final_data_2 = new_data.groupby('SecurityID')
bonds_list_2 = [final_data_2.get_group(x) for x in final_data_2.groups]
bond_dict = {}
bond_spread_change_scaled_list = []
for bond in bonds_list_2:
    bond_spread_change_scaled_list.append((len(bond.G_change.values), bond.SecurityID.iloc[0], bond.G_scale.values))
    bond_dict[bond.SecurityID.iloc[0]] = bond.ModifiedDuration_Plain.iloc[-1]
   #bond_spread_change_scaled_list.append((len(bond.G_change.values), bond.SecurityID.iloc[0], bond.GSpread.values))

In [ ]:
bond_maturity = pd.DataFrame(bond_dict.items(),columns = ['SecurityID','ModifiedDuration'])
bond_maturity

In [ ]:
bond_spread_change_scaled_list

In [ ]:
test_cluster_1 = select_data(33, bond_spread_change_scaled_list)
test_cluster_1_with_spread = select_spread_data(test_cluster_1)
print(len(test_cluster_1_with_spread))

In [ ]:
import matplotlib.pyplot as plt
test_cluster_1
for c in test_cluster_1:
    plt.plot(c[2])
plt.show()

In [ ]:
##perform randon walk test

In [ ]:
som_shape = (2, 2)
from minisom import MiniSom    
som = MiniSom(som_shape[0], som_shape[1], 33, sigma=1.7, learning_rate=.6, activation_distance='euclidean',
            neighborhood_function='gaussian', random_seed=10)
som.pca_weights_init(test_cluster_1_with_spread)
som.train_batch(test_cluster_1_with_spread, 1500, verbose=True)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload

In [ ]:
# each neuron represents a cluster
winner_coordinates = np.array([som.winner(x) for x in test_cluster_1_with_spread]).T
# with np.ravel_multi_index we convert the bidimensional
# coordinates to a monodimensional index
cluster_index = np.ravel_multi_index(winner_coordinates, som_shape)

In [ ]:
###get the cluster
def create_cluster_list(cluster_index):
    temp_1 = []
    for i in np.unique(cluster_index):
        temp_2 = []
        for n in range(0,len(cluster_index)):
            if cluster_index[n] == i:
                temp_2.append(n)
        temp_1.append(temp_2)
    return temp_1
        

In [ ]:
clusters = create_cluster_list(cluster_index)

In [ ]:
def transfertime(x):
    try:
        l = x.split(' ')
        g = l[0].split('-')
        f = datetime.datetime(int(g[0]),int(g[1]),int(g[2]))
        return f
    except:
        pass

In [ ]:
def find_the_duration(x,y):
    try:
        final = y-x
        return final
    except:
        pass

In [ ]:
security_data = pd.read_csv("C:/Users/y437l/OneDrive/MMAI/Capstone/Data/SecurityData - Copy-LAPTOP-OAEJRPE8.csv")
rating_data = process_rating_data(pd.read_csv("C:/Users/y437l/OneDrive/MMAI/Capstone/Data/Rating.csv"))
security_data = security_data.merge(bond_maturity,on=['SecurityID'],how='left')

In [ ]:
clusters_list = grab_clusters(clusters,bond_spread_list,security_data,rating_data)

In [ ]:
len(clusters_list)

In [ ]:
clusters_data_list = transfer_to_list_of_pd(clusters_list)

In [ ]:
clusters_data_list[0]

In [ ]:
clusters_data_list[1]

In [ ]:
clusters_data_list[2]

In [ ]:
clusters_data_list[3]

In [ ]:
clusters_data_list[0].to_csv('test0.csv')

In [ ]:
import matplotlib.pyplot as plt
test_cluster_1
for c in clusters_data_list[3]['SecurityID']:
    plt.plot(new_data[new_data['SecurityID']==c].G_scale.values)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
test_cluster_1
for c in clusters_data_list[1]['SecurityID']:
    plt.plot(new_data[new_data['SecurityID']==c].G_scale.values)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
test_cluster_1
for c in clusters_data_list[0]['SecurityID']:
    plt.plot(new_data[new_data['SecurityID']==c].G_scale.values)
plt.show()
print(len(clusters_data_list[0]))

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
from pandas_profiling import ProfileReport
prof = ProfileReport(clusters_data_list[0])
prof

In [ ]:
prof1 = ProfileReport(clusters_data_list[1])
prof1

In [ ]:
prof3 = ProfileReport(clusters_data_list[3])
prof3

In [ ]:
prof2 = ProfileReport(clusters_data_list[2])
prof2

In [ ]:
#Feature extraction